# Set Up

In [5]:
%pwd


'/pscratch/sd/m/mansisak/memorization/figs'

In [ ]:
!conda list

In [6]:
!module load conda

In [7]:
!conda activate /pscratch/sd/m/mansisak/memorization/env/

In [24]:
!ls /pscratch/sd/m/mansisak/memorization/model_ckpts/EleutherAI_edit/

act	localization_results_108000.csv  mem_seq_pythia-2.8b-deduped
attrib	localization_results_143000.csv  mem_seq_pythia-6.9b-deduped
greedy	localization_results_36000.csv	 slim
hc	localization_results_72000.csv


In [ ]:
ls 

In [16]:
import pandas as pd

# Load in Results

In [19]:
result_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/EleutherAI_edit/"

file_name = "localization_results_72000.csv"

df = pd.read_csv(f'{result_path}{file_name}')

base_stats = df.loc[0].copy()
df = df.drop_duplicates()
df.head()

,model_name,localization_method,ratio,batch_size,epochs,lambda_l1,stop_loss,lr,prompt_len,ig_steps,momentum,weight_decay,step,assess_mem,seed,model_path,results_path,perc,perp,unlearn_set_name
0,EleutherAI/pythia-6.9b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.564356,20.980392,NaN
1,EleutherAI/pythia-6.9b-deduped,act,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.281188,21.607843,mem
2,EleutherAI/pythia-6.9b-deduped,act,0.010,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.164356,23.372549,mem
3,EleutherAI/pythia-6.9b-deduped,act,0.050,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.108911,25.411765,mem
4,EleutherAI/pythia-6.9b-deduped,hc,0.001,32,1,1000,0.1,0.1,32,1,0.9,0.0005,72000,0,0,../../model_ckpts/EleutherAI/pythia-6.9b-deduped,../../model_ckpts/EleutherAI_edit/localization...,0.512871,21.568627,mem
